In [1]:
pip install chromadb openai langchain fastapi uvicorn sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [7]:
import json
import hashlib
import chromadb
from sentence_transformers import SentenceTransformer

# Load a transformer model for embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize ChromaDB with Persistent Storage
client = chromadb.PersistentClient(path="./chromadb")
collection = client.get_or_create_collection(name="Query_builder")

def generate_id(text):
    """Generate a unique ID for each ACL query using SHA256 hash."""
    return hashlib.sha256(text.encode()).hexdigest()

def store_acl_mapping(acl_query, databricks_sql):
    """Store ACL query and its equivalent Databricks SQL in ChromaDB with embeddings."""
    unique_id = generate_id(acl_query)  # Generate a unique ID
    embedding = embedding_model.encode(acl_query).tolist()

    # Avoid duplicate storage
    existing_data = collection.get(ids=[unique_id], include=["documents"])
    if existing_data and existing_data["documents"]:  
        print(f"⚠️ Query already exists, skipping: {acl_query}")
        return  

    collection.add(
        ids=[unique_id],  
        embeddings=[embedding], 
        documents=[acl_query],  # Store ACL query
        metadatas=[{"databricks_sql": databricks_sql}]
    )
    print(f"✅ Stored: {acl_query}")

def load_and_store_json(json_path):
    """Load ACL to Databricks SQL mappings from a JSON file and store in ChromaDB."""
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"❌ Error loading JSON file: {e}")
        return

    for entry in data:
        acl_query = entry["input"].strip()
        databricks_sql = entry["output"].strip()
        store_acl_mapping(acl_query, databricks_sql)
    
    print(f"✅ Finished storing {len(data)} ACL queries in ChromaDB.")

# Provide your JSON file path
json_file_path = "extended_acl_to_databricks_dataset.json"
load_and_store_json(json_file_path)


✅ Stored: AVG Year FROM employees
✅ Stored: SORT departments BY Region ASC
✅ Stored: FILTER orders WHERE Department = 'Sales'
✅ Stored: UNION departments AND employees
✅ Stored: HAVING orders WHERE Quantity < 10
✅ Stored: FILTER orders WHERE Quantity IN (SELECT Quantity FROM sales WHERE Quantity < 10)
✅ Stored: AVG ProductID FROM transactions
✅ Stored: SUM HireDate FROM departments
✅ Stored: LIMIT products TO 100
✅ Stored: WITH cte_39 AS (SELECT Department, Region, ProductID, HireDate, Salary, Year, Quantity, OrderID, Price FROM transactions WHERE Department = 'Sales') SELECT Department, Region, ProductID, HireDate, Salary, Year, Quantity, OrderID, Price FROM cte_39
✅ Stored: SELECT Department, OrderID FROM customers WHERE Salary > 50000
✅ Stored: SELECT Region, OrderID, Year, Quantity, DATE_TRUNC('year', ProductID) AS Year FROM sales
✅ Stored: LIMIT sales TO 20
✅ Stored: FILTER departments WHERE Salary IN (SELECT Salary FROM departments WHERE Region = 'North')
✅ Stored: SORT departmen

In [8]:
import numpy as np

def retrieve_similar_queries(acl_query, top_k=3):
    """Retrieve the most similar stored ACL queries from ChromaDB."""
    embedding = embedding_model.encode(acl_query).tolist()

    # Perform similarity search in ChromaDB
    results = collection.query(
        query_embeddings=[embedding],
        n_results=top_k,  # Get top-k similar queries
        include=["documents", "metadatas"]  # Fetch stored queries & their metadata
    )

    # Extract best matching queries and SQL outputs
    similar_queries = results["documents"][0]  # List of matching ACL queries
    metadata = results["metadatas"][0]  # List of Databricks SQL outputs

    # Handle case where no match is found
    if not similar_queries or not metadata:
        print("⚠️ No similar queries found.")
        return None

    # Return top match
    return metadata[0]["databricks_sql"]  # Best match's Databricks SQL


In [10]:
# Example ACL query to convert
query = "select orders WHERE Region = 'North'"

# Retrieve and print the best matching Databricks SQL query
converted_query = retrieve_similar_queries(query)
print("🔄 Converted Databricks SQL:", converted_query)


🔄 Converted Databricks SQL: SELECT * FROM orders WHERE Region = 'North';


In [12]:
import ollama

def refine_query_with_ollama(acl_query):
    """Refine ACL to Databricks SQL conversion using Ollama model."""
    
    retrieved_sql = retrieve_similar_queries(acl_query)  # Ensure this function is defined
    
    if not retrieved_sql:
        return "⚠️ No matching query found."

    prompt = f"""
    Convert the following ACL query into an optimized Databricks SQL query:
    
    ACL Query: {acl_query}
    
    Based on a similar existing query:
    {retrieved_sql}

    Ensure the output is syntactically correct, optimized, and adheres to best SQL practices.
    """

    try:
        response = ollama.chat(
            model="deepseek-r1",
            messages=[{"role": "user", "content": prompt}],
            options={"temperature": 0}  # Ensure deterministic output
        )
        return response["message"]["content"]
    
    except Exception as e:
        return f"⚠️ Error processing query: {str(e)}"

# Example usage
acl_query = "COUNT orders WHERE Region = 'North'"
databricks_query = refine_query_with_ollama(acl_query)
print("🚀 Final Databricks Query:", databricks_query)


🚀 Final Databricks Query: <think>
Okay, so I need to convert an ACL query into an optimized Databricks SQL query. The original ACL query is "COUNT orders WHERE Region = 'North'". There's also a similar existing query that serves as a reference: "SELECT COUNT(*) FROM orders WHERE Region = 'North';".

First, I should understand what the difference is between these two queries. The ACL version seems to be using just "COUNT orders", which might not specify the column clearly. In SQL terms, it's more precise to use "COUNT(order_id)" or something similar because it specifies exactly which column we're counting.

Looking at the existing query, it uses SELECT COUNT(*) FROM orders WHERE Region = 'North'. This is a standard way in SQL to count all rows that meet a certain condition. So for Databricks, I should probably follow this structure but make sure it's optimized.

Databricks SQL has some optimizations and specific functions. For instance, using "AS" clauses can improve readability. Also, 